In [1]:
import ares
import numpy as np
import matplotlib.pyplot as pl
import distpy

In [2]:
# Independent variables
redshifts = np.array([0.35, 0.875, 1.125, 1.75, 2.25, 2.75])
# MUV = np.arange(-28, -8.8, 0.2)
Ms = np.logspace(8, 12, 50)

# blob 1: the LF. Give it a name, and the function needed to calculate it.
blob_n1 = ['galaxy_smf']
blob_i1 = [('z', redshifts), ('bins', Ms)]
blob_f1 = ['StellarMassFunction']

# # blob 2: the SFE. Same deal.
# blob_n2 = ['fstar']
# blob_i2 = [('z', redshifts), ('Mh', Mh)]
# blob_f2 = ['fstar']

In [3]:
blob_pars = \
{
 'blob_names': [blob_n1],
 'blob_ivars': [blob_i1],
 'blob_funcs': [blob_f1],
 'blob_kwargs': [None],
}

In [4]:
#define the parameters that remain unchanged
base_pars = ares.util.ParameterBundle('emma:model1')
base_pars.update(blob_pars)
base_pars.update({'progress_bar': True})

# This is important!
# base_pars['pop_calib_lum'] = None

In [47]:
free_pars = \
[
    'pq_func_par0[0]',
    'pq_func_par2[0]', 

    #norm
    'pq_func_par0[1]',
    'pq_func_par2[1]', 

    #gamma
    'pq_func_par0[2]',
    'pq_func_par2[2]', 

    #peak mass
    'pq_func_par0[3]',
    'pq_func_par2[3]', 
]

is_log = [False, False, False, False, False, False, False, False]

from distpy.distribution import UniformDistribution
from distpy.distribution import DistributionSet

ps = DistributionSet()
ps.add_distribution(UniformDistribution(0, 4), 'pq_func_par0[0]')
ps.add_distribution(UniformDistribution(-1, 1),  'pq_func_par2[0]')

ps.add_distribution(UniformDistribution(0.001, 2),   'pq_func_par0[1]')
ps.add_distribution(UniformDistribution(-1, 1),  'pq_func_par2[1]')

ps.add_distribution(UniformDistribution(0, .7),   'pq_func_par0[2]')
ps.add_distribution(UniformDistribution(-3, -0.01),  'pq_func_par2[2]')

ps.add_distribution(UniformDistribution(11.4, 12.6),   'pq_func_par0[3]')
ps.add_distribution(UniformDistribution(0, 1),  'pq_func_par2[3]')

In [70]:
#From Moster2010, table 7
logM_0 = 11.88 #(0.01)
mu = 0.019 #(0.002)
N_0 = 0.0282 #(0.0003)
nu = -0.72 #(0.06)
gamma_0 = 0.556 #0.001
gamma_1 = -0.26 #(0.05)
beta_0 = 1.06 #(0.06)
beta_1 = 0.17 #(0.12)

guesses = \
{
    'pq_func_par0[0]': beta_0,
    'pq_func_par2[0]': beta_1, 

    #norm
    'pq_func_par0[1]': N_0,
    'pq_func_par2[1]': nu, 

    #gamma
    'pq_func_par0[2]': gamma_0,
    'pq_func_par2[2]': gamma_1, 

    #peak mass
    'pq_func_par0[3]': logM_0,
    'pq_func_par2[3]': mu, 
}

In [71]:
# Initialize a fitter object and give it the data to be fit
fitter_smf = ares.inference.FitGalaxyPopulation(**base_pars)

fitter_smf.include.append('smf')

# The data can also be provided more explicitly
#I seem to need this or else the run throws error: Must set data by hand! 
# fitter_lf.redshifts = {‘lf’: [5.9]}
fitter_smf.data = 'tomczak2014',  'mortlock2011', 

# print(fitter_smf.data)

lf
not found!
smf
lf
not found!
smf


In [72]:
fitter = ares.inference.ModelFit(**base_pars)
fitter.add_fitter(fitter_smf)

# Establish the object to which we'll pass parameters
from ares.populations.GalaxyHOD import GalaxyHOD
fitter.simulator = GalaxyHOD

In [73]:
# print(fitter.blob_names)
# print(fitter.blob_ivars)
# print(fitter.blob_funcs, fitter.blob_kwargs)


In [74]:
fitter.save_hmf = True  # cache HMF for a speed-up!
fitter.save_psm = True  # cache source SED model (e.g., BPASS, S99)

# Setting this flag to False will make ARES generate new files for each checkpoint.
# 2-D blobs can get large, so this allows us to just download a single
# snapshot or two if we'd like (useful if running on remote machine)
fitter.checkpoint_append = False

fitter.parameters = free_pars
# fitter.is_log = is_log
fitter.prior_set = ps

# In general, the more the merrier (~hundreds)
fitter.nwalkers = 50

# fitter.jitter = [0.1] * len(fitter.parameters)
fitter.jitter = [0.1, 0.05, 0.03, 0.05, 0.1, 0.1, 0.5, 0.05]

fitter.guesses = guesses
# fitter.pops

# Fixing position of walker 0 (parameter pq_func_par0[1])
# Moved from -0.02053383673944127 to 0.026124647013238887
# Fixing position of walker 4 (parameter pq_func_par0[1])
# Moved from -0.007207661454420058 to 0.00786890259922033
# Fixing position of walker 6 (parameter pq_func_par0[1])
# Moved from -0.031256733133269154 to 0.029395641098237016
# Fixing position of walker 21 (parameter pq_func_par0[1])
# Moved from -0.026146336920826605 to 0.04616697670537091
# Fixing position of walker 47 (parameter pq_func_par0[1])
# Moved from -0.0004930245771815507 to 0.00965879123730318
# Fixing position of walker 4 (parameter pq_func_par0[2])
# Moved from 0.7429407496103543 to 0.6082540062193245
# Fixing position of walker 5 (parameter pq_func_par0[2])
# Moved from 0.702542829096455 to 0.6865282131818774
# Fixing position of walker 7 (parameter pq_func_par0[2])
# Moved from 0.7014989339839134 to 0.6465804551427892
# Fixing position of walker 22 (parameter pq_func_par0[2])
# Moved from 0.7058712

In [75]:
# Run the thing
fitter.run('test_smfcal', burn=5, steps=30, save_freq=2, clobber=True)

Saved HaloMassFunction instance to limit I/O.
# Starting burn-in: Thu Jun 18 15:14:33 2020
# Wrote test_smfcal.burn.dd0000.facc.pkl: Thu Jun 18 15:15:34 2020
# Wrote test_smfcal.burn.dd0001.facc.pkl: Thu Jun 18 15:16:10 2020
FAILURE:  {'pq_func_par0[0]': 1.1346832916307468, 'pq_func_par2[0]': 0.3055356930112487, 'pq_func_par0[1]': 0.04800604409284713, 'pq_func_par2[1]': -0.98508576168629, 'pq_func_par0[2]': 0.6660331555491057, 'pq_func_par2[2]': -0.12049150824010797, 'pq_func_par0[3]': 12.10598385290876, 'pq_func_par2[3]': 0.4810944658583315}
Writing data one last time because save_freq > steps.
# Wrote test_smfcal.burn.dd0002.facc.pkl: Thu Jun 18 15:16:26 2020
# Burn-in complete in 113 seconds.
# Fixing position of walker 4 (parameter pq_func_par0[1])
# Moved from -0.006422992891967759 to 0.03304956724599179
# Fixing position of walker 5 (parameter pq_func_par0[1])
# Moved from -0.005914899806349386 to 0.0036246093432313886
# Fixing position of walker 6 (parameter pq_func_par0[1])
# M

In [76]:
anl = ares.analysis.ModelSet('test_smfcal')



##############################################################################################################
####                                         Analysis: Model Set                                          ####
##############################################################################################################
#### ---------------------------------------------------------------------------------------------------- ####
#### Basic Information                                                                                    ####
#### ---------------------------------------------------------------------------------------------------- ####
#### path        : ./                                                                                     ####
#### prefix      : test_smfcal                                                                            ####
#### N-d         : 8                                                                                      ####


In [ ]:
#look at the raw LF samples
# ax = anl.ReconstructedFunction('galaxy_lf', ivar=[6, None], samples='all', color='b', alpha=0.01)

# ax.set_yscale('log')
z = 0.875 
# 0.35, 0.875, 1.125, 1.75, 2.25, 2.75

gpop = ares.analysis.GalaxyPopulation()

ax = anl.ReconstructedFunction('galaxy_smf', ivar=[z, None], samples='all', color='b', alpha=0.01)

# Plot any data within dz=0.1 of z=6
gpop.PlotSMF(z, ax=ax, round_z=0.2)

ax.set_ylim(1e-9, 1)
ax.legend()
pl.show()